In [0]:
#connecting google drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#saving dataset(dowload) file to google drive
!cp /content/a.zip /content/drive/My\ Drive/a.zip 

In [0]:
#retreiving dataset from google drive
!cp /content/drive/My\ Drive/a.zip /content/a.zip

In [0]:
#this is one time process for storing dataset in google drive

In [0]:
#unzip dataset
!unzip a.zip 

In [0]:
#for removing unziped folder due to trubleshooting
!rm -d -r /content/a.zip

In [0]:
# Deep neural network
#convolution neural network
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm


In [0]:
#paths for datafile
train_images_paths="/content/MURA-v1.1/train_image_paths.csv"
#loading csv files
train_paths = pd.read_csv(train_images_paths)
valid_image_paths='/content/MURA-v1.1/valid_image_paths.csv'
valid_paths = pd.read_csv(valid_image_paths)

In [0]:
train_paths.head(4)

In [0]:
valid_paths.head(4)

In [0]:
def diagnose(train_path):
  train_label_diag=111;
  if train_path.find('study1_positive')!=-1 or train_path.find('study2_positive')!=-1 or \
  train_path.find('study3_positive')!=-1 or train_path.find('study4_positive')!=-1:
    train_label_diag=1
  elif train_path.find('study1_negative')!=-1 or train_path.find('study2_negative')!=-1 or \
  train_path.find('study3_negative')!=-1 or train_path.find('study4_negative')!=-1:
    train_label_diag=0
  else:
    train_label_diag=-1
  return train_label_diag

In [0]:
#loading dataset for training from csv file
train_image=[]
train_label_diag=[]
for i in tqdm(range(train_paths.shape[0])):
      if train_paths['MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png'][i].find('XR_ELBOW')!=-1:
        img = image.load_img('/content/'+train_paths['MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png'][i], target_size=(456,512), grayscale=True)
        img = image.img_to_array(img)
        img = img/255
        train_image.append(img)
        train_label_diag.append(diagnose(train_paths['MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png'][i]))

In [0]:
valid_image=[]
valid_label_diag=[]
del img
for i in tqdm(range(valid_paths.shape[0])):
      if valid_paths['MURA-v1.1/valid/XR_WRIST/patient11185/study1_positive/image1.png'][i].find('XR_ELBOW')!=-1:
        img = image.load_img('/content/'+valid_paths['MURA-v1.1/valid/XR_WRIST/patient11185/study1_positive/image1.png'][i], target_size=(456,512), grayscale=True)
        img = image.img_to_array(img)
        img = img/255

       
        valid_image.append(img)
        valid_label_diag.append(diagnose(valid_paths['MURA-v1.1/valid/XR_WRIST/patient11185/study1_positive/image1.png'][i]))

In [0]:
#convirting data to array format for training
x_train=np.array(train_image)
y_train=np.array(train_label_diag)
x_valid=np.array(valid_image)
y_valid=np.array(valid_label_diag)

In [0]:
x_valid.shape

In [0]:

Y_train=to_categorical(y_train)
Y_valid=to_categorical(y_valid)

In [0]:
x_train_image=x_train[:,:,:,0].copy()
x_valid_image=x_valid[:,:,:,0].copy()

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(456,512,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [0]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 454, 510, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 452, 508, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 226, 254, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 252, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 126, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 110, 124, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 55, 62, 64)        0

In [0]:
from keras.models import load_model
from keras.initializers import glorot_uniform

model=load_model('elbow.h5',custom_objects={'GlorotUniform': glorot_uniform()}, compile=False)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 454, 510, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 452, 508, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 226, 254, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 252, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 126, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 110, 124, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 55, 62, 64)        0

In [0]:
x_valid.shape

(1, 456, 512, 1)

In [0]:
model.fit(x_train, Y_train, epochs=8, validation_data=(x_valid, Y_valid))

Train on 5106 samples, validate on 461 samples
Epoch 1/8
5106/5106 [==============================] - 939s 184ms/sample - loss: 0.6597 - acc: 0.6406 - val_loss: 0.6991 - val_acc: 0.6117
Epoch 2/8
5106/5106 [==============================] - 937s 183ms/sample - loss: 0.5968 - acc: 0.6743 - val_loss: 0.6682 - val_acc: 0.6312
Epoch 3/8
5106/5106 [==============================] - 934s 183ms/sample - loss: 0.5722 - acc: 0.7002 - val_loss: 0.6847 - val_acc: 0.6551
Epoch 4/8
5106/5106 [==============================] - 940s 184ms/sample - loss: 0.5310 - acc: 0.7288 - val_loss: 0.6698 - val_acc: 0.6594
Epoch 5/8
5106/5106 [==============================] - 935s 183ms/sample - loss: 0.4904 - acc: 0.7575 - val_loss: 0.7796 - val_acc: 0.6269
Epoch 6/8
5106/5106 [==============================] - 934s 183ms/sample - loss: 0.4420 - acc: 0.7902 - val_loss: 0.8004 - val_acc: 0.6074
Epoch 7/8
5106/5106 [==============================] - 942s 185ms/sample - loss: 0.3729 - acc: 0.8371 - val_loss: 0.904

In [0]:
prediction = model.predict_classes(x_valid)

In [0]:
model.save('model.h5')    

In [0]:
from sklearn.metrics import confusion_matrix
con_mat=confusion_matrix(y_valid, prediction)
print(con_mat)

In [0]:
con_mat_norm = np.around(con_mat.astype('float')/ con_mat.sum(axis=1)[:,np.newaxis],decimals=2)
con_mat_df = pd.DataFrame(con_mat_norm, index = ['negative', 'positive'], columns=['negative','positive'])
con_mat_df


In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
figure = plt.figure()
sns.heatmap(con_mat_df, annot=True, cmap=plt.cm.Blues)
plt.tight_layout()
plt.xlabel('Prediction Labels')
plt.ylabel('True Labels')
plt.show()